In [2]:
#Import needed libraries
import numpy as np
import random
import tensorflow as tf

# Function to check if a player has won
def check_win(player, board):
    for i in range(3):
        if all(board[i, :] == player) or all(board[:, i] == player):
            return True
    if all(np.diag(board) == player) or all(np.diag(np.fliplr(board)) == player):
        return True
    return False

# Function to get a list of available moves
def available_moves(board):
    return [(i, j) for i in range(3) for j in range(3) if board[i, j] == 0]

# Function to choose a move for the agent
def choose_move(board, epsilon, q_values):
    if random.uniform(0, 1) < epsilon:
        return random.choice(available_moves(board))
    else:
        max_q = -float('inf')
        best_move = None
        for move in available_moves(board):
            next_board = board.copy()
            next_board[move] = 1  # Represent 'X' with 1
            state_tensor = tf.convert_to_tensor(next_board[np.newaxis, :], dtype=tf.float32)
            q_value = q_values.predict(state_tensor)[0, move[0] * 3 + move[1]]
            if isinstance(q_value, tf.Tensor):
                q_value = q_value.numpy()

            if q_value > max_q:
                max_q = q_value
                best_move = move

        return best_move

# Function to play the game
def play_game(agent, epsilon):
    board = np.zeros((3, 3), dtype=np.int32)
    player_turn = True

    while True:
        if player_turn:
            print("Current board:")
            print(board)
            row = int(input("Enter the row (0, 1, 2): "))
            col = int(input("Enter the column (0, 1, 2): "))
            if (row, col) not in available_moves(board):
                print("Invalid move. Try again.")
                continue
            board[(row, col)] = 1  # Represent 'X' with 1
        else:
            move = choose_move(board, epsilon, agent)
            print(f"Agent chooses: {move}")
            board[move] = 2  # Represent 'O' with 2

        if check_win(1, board):
            print("Player wins!" if player_turn else "Agent wins!")
            print("Final board:")
            print(board)
            break

        if len(available_moves(board)) == 0:
            print("It's a draw!")
            print("Final board:")
            print(board)
            break

        player_turn = not player_turn  # Switch turns

# Define the Q-network architecture for DQN
def build_dqn_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(3, 3)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(9, activation=None)  # 9 outputs, one for each possible move
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Function to train the agent using Double DQN
def train_double_dqn(agent, target_agent, epsilon, discount_factor, epochs):
    for epoch in range(epochs):
        board = np.zeros((3, 3), dtype=np.int32)
        state_history = []
        player_turn = True

        while True:
            if player_turn:
                move = choose_move(board, epsilon, agent)
                state_history.append((board.copy(), move))
                board[move] = 1  # Represent 'X' with 1
            else:
                move = choose_move(board, epsilon, target_agent)
                board[move] = 2  # Represent 'O' with 2

            if check_win(1, board) or check_win(2, board) or len(available_moves(board)) == 0:
                break

            player_turn = not player_turn  # Switch turns

        rewards = np.zeros(len(state_history))
        rewards[-1] = 1 if check_win(1, board) else -1 if check_win(2, board) else 0

        for i in range(len(state_history) - 2, -1, -1):
            rewards[i] = rewards[i + 1] * discount_factor

        for i, (state, move) in enumerate(state_history):
            state_tensor = tf.convert_to_tensor(state[np.newaxis, :], dtype=tf.float32)
            next_board_tensor = tf.convert_to_tensor(board[np.newaxis, :], dtype=tf.float32) if i + 1 < len(state_history) else None

            with tf.GradientTape() as tape:
                q_values = agent(state_tensor)

                if next_board_tensor is not None:
                    # Use the target Q-network to evaluate the action
                    next_q_values_target = target_agent(next_board_tensor).numpy()[0]
                    chosen_q = q_values[0, move[0] * 3 + move[1]]
                    td_target = rewards[i] + discount_factor * next_q_values_target[np.argmax(q_values)] - chosen_q
                else:
                    # For terminal state, Q-value equals the reward
                    td_target = rewards[i] - q_values[0, move[0] * 3 + move[1]]

                loss = tf.reduce_mean(tf.square(td_target))

            gradients = tape.gradient(loss, agent.trainable_variables)
            agent.optimizer.apply_gradients(zip(gradients, agent.trainable_variables))

    return agent

# Set hyperparameters
epsilon = 0.1
discount_factor = 0.9
epochs = 1000

# Build models
double_dqn_agent = build_dqn_model()

# Train models
double_dqn_target_agent = build_dqn_model()
double_dqn_agent = train_double_dqn(double_dqn_agent, double_dqn_target_agent, epsilon, discount_factor, epochs)

# Play a game with the trained agent
play_game(double_dqn_agent, epsilon)


1/1 [==============================] - 0s 22ms/step
Current board:
[[0 0 0]
 [0 0 0]
 [0 0 0]]
Enter the row (0, 1, 2): 1
Enter the column (0, 1, 2): 2
Agent chooses: (2, 2)
Current board:
[[0 0 0]
 [0 0 1]
 [0 0 2]]
Enter the row (0, 1, 2): 11
Enter the column (0, 1, 2): 2
Invalid move. Try again.
Current board:
[[0 0 0]
 [0 0 1]
 [0 0 2]]
Enter the row (0, 1, 2): 0
Enter the column (0, 1, 2): 1
1/1 [==============================] - 0s 176ms/step
Agent chooses: (1, 0)
Current board:
[[0 1 0]
 [2 0 1]
 [0 0 2]]
Enter the row (0, 1, 2): 2
Enter the column (0, 1, 2): 1
1/1 [==============================] - 0s 23ms/step
Agent chooses: (1, 1)
Current board:
[[0 1 0]
 [2 2 1]
 [0 1 2]]
Enter the row (0, 1, 2): 0
Enter the column (0, 1, 2): 2
1/1 [==============================] - 0s 22ms/step
Agent chooses: (0, 0)
Current board:
[[2 1 1]
 [2 2 1]
 [0 1 2]]
Enter the row (0, 1, 2): 2
Enter the column (0, 1, 2): 0
It's a draw!
Final board:
[[2 1 1]
 [2 2 1]
 [1 1 2]]
